In [1]:
import pandas as pd
import os

In [2]:
# 연도별 엑셀 파일 명 리스트
years = list(range(2014,2024))
file_list = [f"raw_doso/TB_TAT_KAMIS_PRC_MK_TB-{year}.csv" for year in years]
file_list

['raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2014.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2015.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2016.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2017.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2018.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2019.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2020.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2021.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2022.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2023.csv']

In [3]:
# 저장할 df를 저장하는 딕셔너리
dfs = {}

# df에서 농산물 항목 목록
products = ['깐마늘(수입)', '깐마늘(국산)']
# products = ['방울토마토','풋고추','고구마','감자','배추','양배추','시금치','상추','오이','건고추','호박','토마토','무','깐마늘','양파','파','미나리','깻잎','파프리카','팽이버섯','새송이버섯','얼갈이배추','당근','부추','양상추']

# 도매 소매 구분
channels = ['도매', '소매']

# 모든 연도별 파일 순회
for file_name in file_list:
    df = pd.read_csv(f"raw_data/{file_name}", encoding='cp949')
    df.columns = ['가격등록일자','시장고유번호','시장코드','시장명','시도코드','시도명','시군구코드','시군구명','품목코드','품목명','품종코드','품종명','도소매조사구분코드','조사구분명','산물등급코드','산물등급명','품목가격','산지출하단위크기','산지출하단위명','도매출하단위크기','도매출하단위명','소매출하단위크기','소매출하단위명','친환경농산물출하단위크기','친환경농산물출하단위명','할인가격여부','ETL적재일시']
    df = df[['가격등록일자','시장코드','시장명','시도코드','시도명','품목코드','품목명','품종코드','품종명','조사구분명','산물등급코드','산물등급명','품목가격','도매출하단위크기','도매출하단위명','소매출하단위크기','소매출하단위명']]
    df['품목명'] = df['품목명'].astype(str)

    # 각 농산물별로 df 분할
    for product in products:
        product_df = df[df['품목명'] == product] # 해당 농산물에 대한 df

        for channel in channels:
            channel_df = product_df[product_df['조사구분명'] == channel]
            channel_df = channel_df[['가격등록일자','시장코드','시장명','시도코드','시도명','품목코드','품목명','품종코드','품종명','조사구분명','산물등급코드','산물등급명','품목가격',f'{channel}출하단위크기',f'{channel}출하단위명']]

            # 딕셔너리에 농산물 데이터 프레일 저장
            product_channel_key = f"{product}_{channel}"
            if product_channel_key not in dfs:
                dfs[product_channel_key] = channel_df
            else:
                # pd.concat 사용해 df 연결
                dfs[product_channel_key] = pd.concat([dfs[product_channel_key], channel_df])



# 범위 시작 및 종료 날짜
start_date = '2014-01-01'
end_date = '2023-05-10'

# 지정된 범위의 날짜를 포함하는 시리즈 생성
date_series = pd.date_range(start=start_date, end=end_date, freq='D')

# 날짜를 원하는 숫자 형식으로 변환
numeric_dates = date_series.strftime('%Y%m%d').astype(int)

# 숫자 형식의 날짜를 포함하는 데이터프레임 생성
date_df = pd.DataFrame(numeric_dates, columns=['가격등록일자'])

print(date_df)


# 각 농산물별로 새로운 엑셀 파일 저장
output_folder = "output/doso"
for product_channel_key, product_channel_df in dfs.items():
    product_channel_df = pd.merge(date_df,product_channel_df, left_on='가격등록일자', right_on='가격등록일자', how='outer')
    # 필요하다면 인덱스를 리셋하여 저장
    output_df = product_channel_df.reset_index(drop=True)
    output_path = os.path.join(output_folder, f"{product_channel_key}_prices.xlsx")
    output_df.to_excel(output_path, index=False)

/var/folders/7d/d1khjvv95fs1dx3zplr2sfzh0000gn/T/ipykernel_8733/3747483925.py:13: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"raw_data/{file_name}", encoding='cp949')
/var/folders/7d/d1khjvv95fs1dx3zplr2sfzh0000gn/T/ipykernel_8733/3747483925.py:13: DtypeWarning: Columns (17,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"raw_data/{file_name}", encoding='cp949')
/var/folders/7d/d1khjvv95fs1dx3zplr2sfzh0000gn/T/ipykernel_8733/3747483925.py:13: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"raw_data/{file_name}", encoding='cp949')
/var/folders/7d/d1khjvv95fs1dx3zplr2sfzh0000gn/T/ipykernel_8733/3747483925.py:13: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"raw_data/{file_name}", encoding='cp949')
/var/folders/7d/d

        가격등록일자
0     20140101
1     20140102
2     20140103
3     20140104
4     20140105
...        ...
3412  20230506
3413  20230507
3414  20230508
3415  20230509
3416  20230510

[3417 rows x 1 columns]
